In [17]:
import numpy as np
import scipy as sp
import math
import Universality_check as Uc
import Gate_helper
import Swap_generators

### Gate Set (S)

In [18]:
## Fermionic puzzle
Perms_f = Swap_generators.swaps(-1,-1)
SQRT_SWAP_f = Swap_generators.sqrt_swaps(-1,-1)
Quarter_Swap_f = Swap_generators.general_swaps(-1,-1,theta=np.pi/8)
S_f = SQRT_SWAP_f
## Bosonic puzzle
Perms_b = Swap_generators.swaps(1,1)
SQRT_SWAP_b = Swap_generators.sqrt_swaps(1,1)
Quarter_Swap_b = Swap_generators.general_swaps(1,1,theta=np.pi/8)
S_b = SQRT_SWAP_b
## Mixed puzzle
Perms_m = Swap_generators.swaps(-1,1)# we add diagonal swaps here to get universality.
SQRT_SWAP_m = Swap_generators.sqrt_swaps(-1,1)
Quarter_Swap_m = Swap_generators.general_swaps(-1,1,theta=np.pi/8)
S_m = SQRT_SWAP_m


## Step 1 of the algorithm,

For our gate set to be universal we need the center of our group to be the trivial center which is just scalar multiples of the identity. If there is a non identity element in the center than that element will commute with everything in our gate set and can thus not be synthesised by a sequence of gates. If the center is nontrivial than the groups may still be infinite but they cannot be universal.

In [19]:
print("Center is trivial for fermionic puzzle:",Uc.check_center(S_f))
print("Center is trivial for bosonic puzzle:",Uc.check_center(S_b))
print("Center is trivial for mixed puzzle:",Uc.check_center(S_m))

Center is trivial for fermionic puzzle: False
Center is trivial for bosonic puzzle: False
Center is trivial for mixed puzzle: False


### Step 2 of the algorithm

This code determines the span of our gate set is infinite or finite. It does this by attempting to find an element that can be reached from our gate set that is in a ball of appropriate radius that is not in the center of the group. Note that if this check passes, but the previous check fails, that the span of $S$ is infinite, but not all of $SU(6)$, meaning it is not universal.  

In [20]:
# using a much smaller value for N than is required to be thourough since i want this to actually run 
N_SU6 = 100  #36398100 # upper bound for N

print("Fermionic puzzle is infinite:",not Uc.check_finite(S_f,N_SU6,10,verbose=False))
print("Bosonic puzzle is infinite:",not Uc.check_finite(S_b,N_SU6,10,verbose=False))
print("Mixed puzzle is infinite:",not Uc.check_finite(S_m,N_SU6,10,verbose=False))

Fermionic puzzle is infinite: True
Bosonic puzzle is infinite: True
Mixed puzzle is infinite: True


## Or all together we can check for universality


In [21]:
print("Fermionic puzzle is universal:",Uc.check_universal(S_f))
print("Bosonic puzzle is universal:",Uc.check_universal(S_b))
print("Mixed puzzle is universal:",Uc.check_universal(S_m))

Fermionic puzzle is universal: False
Bosonic puzzle is universal: False
Mixed puzzle is universal: False


# Puzzle variants that achieve universality

## 1) Half Swaps with and an S gate

In [22]:
## add S_gate to fermionic gate set,
S_f1 = [G for G in SQRT_SWAP_f]
S_f1.append(Gate_helper.S_gate(6))

S_b1 = [G for G in SQRT_SWAP_b]
S_b1.append(Gate_helper.S_gate(6))

S_m1 = [G for G in SQRT_SWAP_m]
S_m1.append(Gate_helper.S_gate(6))

In [23]:
print("With S gate the fermionic puzzle is universal:",Uc.check_universal(S_f1))
print("With S gate the bosonic puzzle is universal:",Uc.check_universal(S_b1))
print("With S gate the mixed puzzle is universal:",Uc.check_universal(S_m1))

With S gate the fermionic puzzle is universal: True
With S gate the bosonic puzzle is universal: True
With S gate the mixed puzzle is universal: True


## 2) Add in Diagonal swaps so the puzzle allows all to all swaps

In [24]:
# TODO: Fix this error
Perms_f2 = Swap_generators.swaps(-1,-1,diagonals=True)
SQRT_SWAP_f2 = Swap_generators.sqrt_swaps(-1,-1,diagonals=True)
S_f2 = SQRT_SWAP_f2

Perms_b2 = Swap_generators.swaps(1,1,diagonals=True)
SQRT_SWAP_b2 = Swap_generators.sqrt_swaps(1,1,diagonals=True)
S_b2 = SQRT_SWAP_b2

Perms_m2 = Swap_generators.swaps(-1,1,diagonals=True)
SQRT_SWAP_m2 = Swap_generators.sqrt_swaps(-1,1,diagonals=False)
S_m2 = SQRT_SWAP_m2
S_m2.append(Perms_m2[-1])
S_m2.append(Perms_m2[-2])

In [25]:
print("With diagonal swaps the fermionic puzzle is universal:",Uc.check_universal(S_f2))
print("With diagonal swaps the bosonic puzzle is universal:",Uc.check_universal(S_b2))
print("With diagonal swaps the mixed puzzle is universal:",Uc.check_universal(S_m2))

With diagonal swaps the fermionic puzzle is universal: False
With diagonal swaps the bosonic puzzle is universal: False
With diagonal swaps the mixed puzzle is universal: True


### Quarter Swaps and Eight Swaps

In [26]:
S_f3 = Quarter_Swap_f
S_b3 = Quarter_Swap_b
S_m3 = Quarter_Swap_m

Eighth_Swap_f = Swap_generators.general_swaps(-1,-1,theta=np.pi/16)
Eighth_Swap_b = Swap_generators.general_swaps(1,1,theta=np.pi/16)
Eighth_Swap_m = Swap_generators.general_swaps(-1,1,theta=np.pi/16)

S_f4 = Eighth_Swap_f
S_b4 = Eighth_Swap_b
S_m4 = Eighth_Swap_m

In [27]:
print("With quarter swaps the fermionic puzzle is universal:",Uc.check_universal(S_f3))
print("With quarter swaps the bosonic puzzle is universal:",Uc.check_universal(S_b3))
print("With quarter swaps the mixed puzzle is universal:",Uc.check_universal(S_m3))
print('------------------------------------------------------')
print("With eighth swaps the fermionic puzzle is universal:",Uc.check_universal(S_f4))
print("With eighth swaps the bosonic puzzle is universal:",Uc.check_universal(S_b4))
print("With eighth swaps the mixed puzzle is universal:",Uc.check_universal(S_m4))

With quarter swaps the fermionic puzzle is universal: False
With quarter swaps the bosonic puzzle is universal: False
With quarter swaps the mixed puzzle is universal: False
------------------------------------------------------
With eighth swaps the fermionic puzzle is universal: False
With eighth swaps the bosonic puzzle is universal: False
With eighth swaps the mixed puzzle is universal: False


# One simple addition that achieves universality

In [28]:
P = np.diag([1,1,1,1,1,-1])


S_f5 = [G for G in SQRT_SWAP_f]
S_f5.append(P)

S_b5 = [G for G in SQRT_SWAP_b]
S_b5.append(P)

S_m5 = [G for G in SQRT_SWAP_m]
S_m5.append(P)

In [29]:
print("Fermions with phase gate:",Uc.check_universal(S_f5))
print("Bosons with phase gate:",Uc.check_universal(S_b5))
print("Mixed puzzle with phast gate:",Uc.check_universal(S_m5))

Fermions with phase gate: True
Bosons with phase gate: True
Mixed puzzle with phast gate: True
